Imports

In [1]:
#import os
#os.environ["KERAS_BACKEND"] = "jax"
import keras

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm, naive_bayes, neighbors, model_selection, ensemble, decomposition, metrics, calibration, linear_model
import numpy as np
import tensorflow as tf
import keras_hub
import glob
import time
import json
import gc
import itertools
from joblib import dump

2026-02-06 15:48:50.691329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770392931.019944      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770392931.111896      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Data input and execution parameters

In [2]:
paths = ["/kaggle/input/data-cleaned-10/data_cleaned/data_cleaned_10/AFS*.csv", "/kaggle/input/data-cleaned-10/data_cleaned/data_cleaned_10/NZFS*.csv", "/kaggle/input/data-cleaned-10/data_cleaned/data_cleaned_10/ASL*.csv"]
all_files = []
for path in paths:
  all_files.extend(glob.glob(path))
all_files

full_df = pd.DataFrame(columns=[])

for filename in all_files:
    temp = pd.read_csv(filename)
    temp["species"] = filename.split("/")[6].split("_")[0]
    full_df = pd.concat([full_df, temp], join = "outer")
df = full_df.reset_index(drop=True)
df["species"] = df["species"].map({"AFS" : 0, "ASL" : 1, "BLKI" : 2, "cat" : 3, "dingo" : 4, "duck" : 5, "goat" : 6, "ibex" : 7, "NZFS" : 8, "sheep" : 9, "squirrel" : 10, "stork" : 11, "TBMU" : 12})

sample_name = "temp"
sample_rate = 10 #Sample rate in Hz
time_window = 5 #Amount of seconds to look back
transfer = True #Should transfer learning be done
use_species = True #Will there be a species index included
normalized = True #Use normalized input variables (mean = 0, std = 1)
max_samples = 100000 #Max seconds of data to include from each species (Will be multplied by sample rate)
del full_df #Used to clean up ram
del temp #Used to clean up ram
gc.collect()

30

In [3]:
for species_id in df["species"].unique():
    ratio = (max_samples * sample_rate) / len(df[df["species"] == species_id].index)
    if ratio < 1: #Undersamples over represented animals
        for segment_id in df[df["species"] == species_id]["segment_id"].unique():
            to_del = df[df["segment_id"] == segment_id]
            to_del_index = to_del.tail(int((1-ratio) * len(to_del.index))).index
            df.drop(to_del_index, axis=0, inplace=True)
df = df.reset_index(drop=True)

In [4]:
animal_ids = df["species"]
segment_ids = df["segment_id"]

X = df.drop(["species", "segment_id", "label", "label_short", "label_long", "Unnamed: 0"], axis = 1)
y_short = df["label_short"]
y_long = df["label_long"]
Y = y_short #Variable renaming for easy use throughout code

if normalized:
    X = (X - X.mean()) / X.std()

if use_species:
    X = X.join(animal_ids)

In [5]:
X

,x,y,z,species
0,1.236916,-0.363895,0.291366,0
1,1.348840,-0.571227,0.140651,0
2,1.266762,-0.614259,0.111943,0
3,1.266762,-0.610347,0.090413,0
4,1.281685,-0.567315,0.068882,0
...,...,...,...,...
417788,0.598948,0.351989,1.321251,1
417789,-0.501639,-0.168298,1.080825,1
417790,-0.087520,-1.603978,0.564088,1
417791,0.699680,-0.551668,1.066471,1


In [6]:
#Use this for ram cleanup as needed
del df
gc.collect()

0

Training and Evalutation

In [7]:
def compute_tss_keras(model, filtered_y, filtered_preds, behaviors, subtitle = "", epochs = 10, split = 0, species_name = "NA"):
    #Following code used to display and save a formatted matplotlib confusion matrix
    #fig, ax = plt.subplots(figsize=(8, 8), dpi = 160,)
    #metrics.ConfusionMatrixDisplay.from_predictions(filtered_y, filtered_preds, ax = ax) #Fix sizing...
    #fig.suptitle(f"{model.name} on {species_name}")
    #plt.title(subtitle)
    #fig.xticks(rotation=90)
    #fig.tight_layout()
    #fig.savefig(f"Matrix_{model.name}_{subtitle}_{split}_{species_name}")
    #fig.show()

    y_actu = pd.Categorical(filtered_y, categories = behaviors)
    y_pred = pd.Categorical(filtered_preds, categories = behaviors)
    df_conf = pd.crosstab(y_actu, y_pred, dropna=False)
    df_conf.to_csv(f"Matrix_{model.name}_{subtitle}_{split}_{species_name}.csv")
    to_save = {"model" : model.name, "Train" : subtitle, "Samples" : len(filtered_y), "Accuracy" : np.mean(filtered_y == filtered_preds), "Epochs" : epochs, "species" : species_name, "TSS" : {}, "F1" : {}, "Precision" : {}, "Recall" : {}}
    for behavior in behaviors:
        target = filtered_y == behavior
        behavior_preds = filtered_preds == behavior
        tempDF = pd.DataFrame(target, columns = ["class.a"])
        tempDF["preds"] = behavior_preds
        tempDF["cat"] = "TN"
        tempDF.loc[(tempDF["class.a"] == True) & (tempDF["preds"] == True), "cat"] = "TP"
        tempDF.loc[(tempDF["class.a"] == False) & (tempDF["preds"] == True), "cat"] = "FP"
        tempDF.loc[(tempDF["class.a"] == True) & (tempDF["preds"] == False), "cat"] = "FN"
        tp = (tempDF["cat"] == "TP").sum()
        tn = (tempDF["cat"] == "TN").sum()
        fn = (tempDF["cat"] == "FN").sum()
        fp = (tempDF["cat"] == "FP").sum()
        
        if tp+fn == 0:
            a = 0
        else:
            a = (tp/(tp+fn))
        if tn+fp == 0:
            b = 0
        else:
            b = (tn/(tn+fp))
        
        tss = a+b - 1
        if tp+fp == 0:
            precision = 0
        else:
            precision = tp/(tp+fp)
        if tp+fn == 0:
            recall = 0
        else:
            recall = tp/(tp+fn)
        if tp+fp+fn == 0:
            f1 = 0
        else:
            f1 = (2*tp)/((2*tp)+fp+fn)
            
        to_save["TSS"][behavior] = tss
        to_save["F1"][behavior] = f1
        to_save["Precision"][behavior] = precision
        to_save["Recall"][behavior] = recall
    with open(f"{model.name}_{subtitle}_{split}_{species_name}.json", 'w') as fp:
        json.dump(to_save, fp)


def compute_tss(model, filtered_y, filtered_preds, behaviors, subtitle = "", split = 0, species_name = "NA"):
    #Following code used to display and save a formatted matplotlib confusion matrix
    #fig, ax = plt.subplots(figsize=(8, 8), dpi = 160,)
    #metrics.ConfusionMatrixDisplay.from_predictions(filtered_y, filtered_preds, ax = ax) #Fix sizing...
    #fig.suptitle(f"{model.name} on {species_name}")
    #plt.title(subtitle)
    #fig.xticks(rotation=90)
    #fig.tight_layout()
    #fig.savefig(f"Matrix_{model.name}_{subtitle}_{split}_{species_name}")
    #fig.show()

    y_actu = pd.Categorical(filtered_y, categories = behaviors)
    y_pred = pd.Categorical(filtered_preds, categories = behaviors)
    df_conf = pd.crosstab(y_actu, y_pred, dropna=False)
    df_conf.to_csv(f"Matrix_{type(model).__name__}_{subtitle}_{split}_{species_name}.csv")
    to_save = {"model" : type(model).__name__, "Train" : subtitle, "Samples" : len(filtered_y), "Accuracy" : np.mean(filtered_y == filtered_preds), "species" : species_name, "TSS" : {}, "F1" : {}, "Precision" : {}, "Recall" : {}}
    for behavior in behaviors:
        target = filtered_y == behavior
        behavior_preds = filtered_preds == behavior
        tempDF = pd.DataFrame(target, columns = ["class.a"])
        tempDF["preds"] = behavior_preds
        tempDF["cat"] = "TN"
        tempDF.loc[(tempDF["class.a"] == True) & (tempDF["preds"] == True), "cat"] = "TP"
        tempDF.loc[(tempDF["class.a"] == False) & (tempDF["preds"] == True), "cat"] = "FP"
        tempDF.loc[(tempDF["class.a"] == True) & (tempDF["preds"] == False), "cat"] = "FN"
        tp = (tempDF["cat"] == "TP").sum()
        tn = (tempDF["cat"] == "TN").sum()
        fn = (tempDF["cat"] == "FN").sum()
        fp = (tempDF["cat"] == "FP").sum()
        
        if tp+fn == 0:
            a = 0
        else:
            a = (tp/(tp+fn))
        if tn+fp == 0:
            b = 0
        else:
            b = (tn/(tn+fp))
        
        tss = a+b - 1
        if tp+fp == 0:
            precision = 0
        else:
            precision = tp/(tp+fp)
        if tp+fn == 0:
            recall = 0
        else:
            recall = tp/(tp+fn)
        if tp+fp+fn == 0:
            f1 = 0
        else:
            f1 = (2*tp)/((2*tp)+fp+fn)
            
        to_save["TSS"][behavior] = tss
        to_save["F1"][behavior] = f1
        to_save["Precision"][behavior] = precision
        to_save["Recall"][behavior] = recall
    with open(f"{type(model).__name__}_{subtitle}_{split}_{species_name}.json", 'w') as fp:
        json.dump(to_save, fp)

In [8]:
def train_models_keras(model_architecutres, data_set, Y, num_features, splits = None, num_splits = 5, time_step = 1, batch_size = 64, epochs = 100, timed = False, threshold = 0, species = False, species_name = "NA", cache = True, val_freq = 100, transfer = False):
    behaviors = sorted(Y.unique())
    num_behaviors = len(behaviors)
    for i in range(len(model_architecutres)):
        cur_model = model_architecutres[i](num_features, time_step, len(behaviors))
        input_size = cur_model.get_config()["layers"][0]["config"]["batch_shape"]
        if (len(input_size) == 2) and (input_size[1] != (num_features * time_step) and (not species)):
            print(f"The model expected inputs of {cur_model.name} does not match the given inputs")
            print(f"Expected inputs: {input_size[1]}")
            print(f"Given inputs: {num_features * time_step}")
            return
        elif (len(input_size) == 2) and (input_size[1] != ((num_features-1) * time_step + 1)) and (species):
            print(f"The model expected inputs of {cur_model.name} does not match the given inputs")
            print(f"Expected inputs: {input_size[1]}")
            print(f"Given inputs: {((num_features-1) * time_step + 1)}")
            return
        elif (len(input_size) == 3) and (input_size[1] != time_step) and (input_size[2] != num_features):
            print(f"The model expected inputs of {cur_model.name} does not match the given inputs")
            print(f"Expected inputs: {input_size[1]}, {input_size[2]}")
            print(f"Given inputs: {time_step}, {num_features}")
            return
        elif (len(input_size) > 3) :
            print("Too many channels requsted for model input")
            print(f"Expected inputs: {input_size}")
            print(f"Given inputs: {time_step}, {num_features}")
            return
    
    statistics = []
    for i in range(len(model_architecutres)):
        statistics.append([0, 0, pd.DataFrame(columns = Y.unique()), pd.DataFrame(columns = Y.unique())])
    
    if (splits == None) or (splits == []): #Expecting a zipped list of indicies i.e. ((train indices 1, test indicies 1), (train indices 2, test indicies 2), etc.)
        #If user does not provide splits we take num_splits to make stratified splits via skf
        skf = model_selection.StratifiedKFold(n_splits=num_splits)
        splits = []
        for train, test in skf.split(np.zeros(len(Y.index)), Y):
            splits.append((-1, train, test))
    else:
        num_splits = len(splits)
    
    i = 0
    for (animal_id, train_index, test_index) in splits:
        keys_tensor = tf.constant(train_index, dtype = np.int64)
        vals_tensor = tf.ones_like(keys_tensor)  # Ones will be casted to True.
        
        table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor),
        default_value=0)  # If index not in table, return 0.
        
        #This massive things just looks up the index in the table
        #If the index is present, return 1, else return 0. It casts those to bools, the maps the output data to remove the index info. Then we rebatch.

        if cache == True:
            #If we cache the results then shuffling won't be a problem
            filtered_ds_train = data_set.filter(lambda x, y: tf.cast(table.lookup(y[0]), tf.bool)).map(lambda a, b: (a, b[1:])).shuffle(100000).batch(batch_size)
        else:
            #If we don't cache shuffling causes the train and test sets to get mismatched
            filtered_ds_train = data_set.filter(lambda x, y: tf.cast(table.lookup(y[0]), tf.bool)).map(lambda a, b: (a, b[1:])).batch(batch_size)
        
        #We do this over for the test indicies (could use ~tf.cast but this allows for the possibility of not using all the data on a split)
        keys_tensor = tf.constant(test_index, dtype = np.int64)
        vals_tensor = tf.ones_like(keys_tensor)
        
        table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor),
        default_value=0)
        filtered_ds_test = data_set.filter(lambda x, y: tf.cast(table.lookup(y[0]), tf.bool)).map(lambda a, b: (a, b[1:])).batch(batch_size)
        
        if (len(input_size) == 2): #Need to hold all required data set sizes for all models
            data_set_shaped_train = filtered_ds_train.map(lambda k, v: (tf.ensure_shape(k, [None, time_step, num_features]), tf.ensure_shape(v, [None, num_behaviors])), num_parallel_calls=tf.data.AUTOTUNE)
            # Only flatten if the model asks for it flattened
            if species:
                data_set_train = data_set_shaped_train.map(lambda k, v: (tf.concat([tf.reshape(k[:, :, :num_features-1], [-1, (num_features-1) * time_step]), tf.reshape(k[:,0,num_features-1], (-1, 1))], 1), v), num_parallel_calls=tf.data.AUTOTUNE)
            else:
                data_set_train = data_set_shaped_train.map(lambda k, v: (tf.reshape(k, [-1, num_features * time_step]), v), num_parallel_calls=tf.data.AUTOTUNE)

            data_set_shaped_test = filtered_ds_test.map(lambda k, v: (tf.ensure_shape(k, [None, time_step, num_features]), tf.ensure_shape(v, [None, num_behaviors])), num_parallel_calls=tf.data.AUTOTUNE)

            if species:
                data_set_test = data_set_shaped_test.map(lambda k, v: (tf.concat([tf.reshape(k[:, :, :num_features-1], [-1, (num_features-1) * time_step]), tf.reshape(k[:,0,num_features-1], (-1, 1))], 1), v), num_parallel_calls=tf.data.AUTOTUNE)
            else:
                data_set_test = data_set_shaped_test.map(lambda k, v: (tf.reshape(k, [-1, num_features * time_step]), v), num_parallel_calls=tf.data.AUTOTUNE)#.cache()
        
        else:
            data_set_train = filtered_ds_train.map(lambda k, v: (tf.ensure_shape(k, [None, time_step, num_features]), tf.ensure_shape(v, [None, num_behaviors])), num_parallel_calls=tf.data.AUTOTUNE)
            data_set_test = filtered_ds_test.map(lambda k, v: (tf.ensure_shape(k, [None, time_step, num_features]), tf.ensure_shape(v, [None, num_behaviors])),num_parallel_calls=tf.data.AUTOTUNE)

        if cache == True:
            data_set_train = data_set_train.cache()
            data_set_train_shuffled = data_set_train
            #data_set_test = data_set_test.cache() #Reduce memory usage by only caching the train set and minimizing test set usage.
        else:
            data_set_train_shuffled = data_set_train.shuffle(1000, reshuffle_each_iteration=True)
        data_set_train = data_set_train.prefetch(tf.data.AUTOTUNE)
        data_set_test = data_set_test.prefetch(tf.data.AUTOTUNE)
        
        for j in range(len(model_architecutres)):
            if timed:
                start = time.process_time()
            
            model = model_architecutres[j](num_features, time_step, len(behaviors))
            model.fit(data_set_train_shuffled, epochs=epochs, validation_data = data_set_test, validation_freq = val_freq, verbose = 2)
            model.save(f"{model.name}_{species_name}_{i}_{animal_id}.keras")
            
            if timed:
                print("Time taken for Train: ")
                print(time.process_time() - start)
                start = time.process_time()
            
            if timed:
                start = time.process_time()
            model_preds_probs_train = model.predict(data_set_train, verbose = 2)
            if timed:
                print("Time taken to predict: ")
                print(time.process_time() - start)
                start = time.process_time()
            
            filter_train = np.nonzero(np.max(model_preds_probs_train, axis = 1) > threshold)
            filtered_probs_train = np.take(model_preds_probs_train, filter_train[0], axis = 0)
            if len(model_preds_probs_train) > len(filtered_probs_train):
                print(f"Removed {len(model_preds_probs_train)-len(filtered_probs_train)} estimates")
            
            model_preds_train = np.argmax(filtered_probs_train, axis = 1)
            mapped_preds_train = np.array([behaviors[x] for x in model_preds_train])
            filtered_y_train = np.argmax(np.concatenate([y for _, y in data_set_train], axis=0), axis = 1).take(filter_train[0])
            filtered_y_train = np.array([behaviors[y] for y in filtered_y_train])
            
            if timed:
                start = time.process_time()
            model_preds_probs_test = model.predict(data_set_test, verbose = 2)
            if timed:
                print("Time taken to predict: ")
                print(time.process_time() - start)
                start = time.process_time()

            filter_test = np.nonzero(np.max(model_preds_probs_test, axis = 1) > threshold)
            filtered_probs_test = np.take(model_preds_probs_test, filter_test[0], axis = 0)
            if len(model_preds_probs_test) > len(filtered_probs_test):
                print(f"Removed {len(model_preds_probs_test)-len(filtered_probs_test)} estimates")
            
            model_preds_test = np.argmax(filtered_probs_test, axis = 1)
            mapped_preds_test = np.array([behaviors[x] for x in model_preds_test])
            filtered_y_test = np.argmax(np.concatenate([y for _, y in data_set_test], axis=0), axis = 1).take(filter_test[0])
            filtered_y_test = np.array([behaviors[y] for y in filtered_y_test])
            
            statistics[j][0] = statistics[j][0] + np.sum(mapped_preds_train == filtered_y_train) / num_splits
            statistics[j][1] = statistics[j][1] + np.sum(mapped_preds_test == filtered_y_test) / num_splits
            
            compute_tss_keras(model, filtered_y_train, mapped_preds_train, Y.unique(), "Training", epochs, i, f"{species_name}_{animal_id}")
            compute_tss_keras(model, filtered_y_test, mapped_preds_test, Y.unique(), "Testing", epochs, i, f"{species_name}_{animal_id}")

            if transfer:
                inner = keras.Model(inputs = model.layers[1].input, outputs = model.layers[-2].output)
                inner.trainable = False
                if species:
                    inner.layers[3].trainable = True
                inputs = keras.Input(shape=model.get_config()["layers"][0]["config"]["batch_shape"][1:])
                x = inner(inputs, training = False)
                x = keras.layers.Dense(64, activation="relu")(x)
                x = keras.layers.Dropout(.2)(x)
                outputs = keras.layers.Dense(len(behaviors), activation="softmax")(x)
                final = keras.Model(inputs=inputs, outputs=outputs, name = f"{model.name}_transfer")
                final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])

                data_set_transfer = data_set_test.shard(10, 0).cache() #Take 10% of the test data to retrain (1/n %)

                final.fit(data_set_transfer, epochs=int(epochs/2), validation_data = data_set_test, validation_freq = val_freq, verbose = 2)
                final.save(f"{final.name}_{species_name}_{i}.keras")
                model_preds_probs_train = final.predict(data_set_test, verbose = 2)
                filter_train = np.nonzero(np.max(model_preds_probs_train, axis = 1) > threshold)
                filtered_probs_train = np.take(model_preds_probs_train, filter_train[0], axis = 0)
                if len(model_preds_probs_train) > len(filtered_probs_train):
                    print(f"Removed {len(model_preds_probs_train)-len(filtered_probs_train)} estimates")
                
                model_preds_train = np.argmax(filtered_probs_train, axis = 1)
                mapped_preds_train = np.array([behaviors[x] for x in model_preds_train])
                filtered_y_train = np.argmax(np.concatenate([y for _, y in data_set_test], axis=0), axis = 1).take(filter_train[0])
                filtered_y_train = np.array([behaviors[y] for y in filtered_y_train])
                
                compute_tss_keras(final, filtered_y_train, mapped_preds_train, Y.unique(), "Transfer", epochs, i, f"{species_name}_{animal_id}")
                
        i = i + 1
    
    return statistics

In [9]:
def train_models(model_architecutres, data_set, Y, num_features, splits = None, num_splits = 5, time_step = 1, timed = False, threshold = 0, species = False, species_name = "NA", species_categories = []):
    """ Statistics are reported in order of modesl w/ [train acc, test acc, train tss, test tss]"""
    
    behaviors = sorted(Y.unique())
    num_behaviors = len(behaviors)

    statistics = []
    for model in model_architecutres:
        statistics.append([0, 0, pd.DataFrame(columns = Y.unique()), pd.DataFrame(columns = Y.unique())])
    
    if (splits == None) or (splits == []): #Expecting a zipped list of indicies i.e. ((train indices 1, test indicies 1), (train indices 2, test indicies 2), etc.)
        #If user does not provide splits we take num_splits to make stratified splits via skf
        skf = model_selection.StratifiedKFold(n_splits=num_splits)
        splits = []
        for train, test in skf.split(np.zeros(len(Y.index)), Y):
            splits.append((-1, train, test))
        else:
            num_splits = len(splits)
            
    i=0
    for (animal_id, train_index, test_index) in splits:
        if timed:
            start = time.process_time()
    
        keys_tensor = tf.constant(train_index, dtype = np.int64)
        vals_tensor = tf.ones_like(keys_tensor)  # Ones will be casted to True.
        
        table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor),
        default_value=0)  # If index not in table, return 0.
        
        #This massive things just looks up the index in the table
        #If the index is present, return 1, else return 0. It casts those to bools, the maps the output data to remove the index info. Then we rebatch.
        
        filtered_ds_train = data_set.filter(lambda x, y: tf.cast(table.lookup(y[0]), tf.bool)).map(lambda a, b: (a, b[1:]))
        
        #We do this over for the test indicies (could use ~tf.cast but this allows for the possibility of not using all the data on a split)
        keys_tensor = tf.constant(test_index, dtype = np.int64)
        vals_tensor = tf.ones_like(keys_tensor)
        
        table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor),
        default_value=0)
        filtered_ds_test = data_set.filter(lambda x, y: tf.cast(table.lookup(y[0]), tf.bool)).map(lambda a, b: (a, b[1:]))
        
        data_set_shaped_train = filtered_ds_train.map(lambda k, v: (tf.ensure_shape(k, [time_step, num_features]), tf.ensure_shape(v, [num_behaviors])))
        # Only flatten if the model asks for it flattened
        if species:
            data_set_train = data_set_shaped_train.map(lambda k, v: (tf.concat([tf.reshape(k[:, :num_features-1], [-1, (num_features-1) * time_step]), tf.reshape(k[0,num_features-1], (-1, 1))], 1), v), num_parallel_calls=tf.data.AUTOTUNE)
        else:
            data_set_train = data_set_shaped_train.map(lambda k, v: (tf.reshape(k, [-1, num_features * time_step]), v))
        
        data_set_shaped_test = filtered_ds_test.map(lambda k, v: (tf.ensure_shape(k, [time_step, num_features]), tf.ensure_shape(v, [num_behaviors])))
        # Only flatten if the model asks for it flattened
        if species:
            data_set_test = data_set_shaped_test.map(lambda k, v: (tf.concat([tf.reshape(k[:, :num_features-1], [-1, (num_features-1) * time_step]), tf.reshape(k[0,num_features-1], (-1, 1))], 1), v), num_parallel_calls=tf.data.AUTOTUNE)
        else:
            data_set_test = data_set_shaped_test.map(lambda k, v: (tf.reshape(k, [-1, num_features * time_step]), v))
        
        if timed:
            print("Time taken for tensorflow processing: ")
            print(time.process_time() - start)
            start = time.process_time()
        
        #Scikit learn can't use tensorflow datasets
        if species:
            train_x = []
            train_y = []
            train_species = []
            for x, y in data_set_train:
                train_x.append(x[:, :(time_step) * (num_features-1)])
                train_y.append(behaviors[np.argmax(y.numpy())])
                train_species.append(x[0, (time_step) * (num_features-1)].numpy())

            train_x = np.concatenate(train_x, axis = 0)
            train_x = np.concatenate([train_x, pd.get_dummies(pd.Categorical(train_species, categories = species_categories)).to_numpy()], axis = 1)
            train_y = np.array(train_y)
        else:
            train_x = []
            train_y = []
            for x, y in data_set_train:
                train_x.append(x)
                train_y.append(behaviors[np.argmax(y.numpy())])
    
            train_x = np.concatenate(train_x, axis = 0)
            train_y = np.array(train_y)
            
        if species:
            test_x = []
            test_y = []
            test_species = []
            for x, y in data_set_test:
                test_x.append(x[:, :(time_step) * (num_features-1)])
                test_y.append(behaviors[np.argmax(y.numpy())])
                test_species.append(x[0, (time_step) * (num_features-1)].numpy())
                
            test_x = np.concatenate(test_x, axis = 0)
            test_x = np.concatenate([test_x, pd.get_dummies(pd.Categorical(test_species, categories = species_categories)).to_numpy()], axis = 1)
            test_y = np.array(test_y)

        else:
            test_x = []
            test_y = []
            for x, y in data_set_test:
                test_x.append(x)
                test_y.append(behaviors[np.argmax(y.numpy())])
            
            test_x = np.concatenate(test_x, axis = 0)
            test_y = np.array(test_y)
        
        if timed:
            print("Time taken for np concatenation: ")
            print(time.process_time() - start)
            start = time.process_time()
        print("Finished data processing")
        for j in range(len(model_architecutres)):
        
            model = model_architecutres[j]
            model.fit(train_x, train_y)
            with open(f"{type(model).__name__}_{species_name}_{i}.pkl", "wb") as f:
                dump(model, f, protocol=5)
            if timed:
                print("Time taken to fit: ")
                print(time.process_time() - start)
                start = time.process_time()
            #Produce the model predictions once for test & train
            
            if timed:
                start = time.process_time()
            model_preds_probs_train = model.predict_proba(train_x)
            if timed:
                print("Time taken to predict: ")
                print(time.process_time() - start)
                start = time.process_time()
            
            filter_train = np.nonzero(np.max(model_preds_probs_train, axis = 1) > threshold)
            filtered_probs_train = model_preds_probs_train[filter_train]
            if len(model_preds_probs_train) > len(filtered_probs_train):
                print(f"Removed {len(model_preds_probs_train)-len(filtered_probs_train)} estimates")
            
            model_preds_train = np.argmax(filtered_probs_train, axis = 1)
            classes = model.classes_
            mapped_preds_train = np.array([classes[x] for x in model_preds_train])
            filtered_y_train = train_y.take(filter_train[0])
            
            if timed:
                start = time.process_time()
            model_preds_probs_test = model.predict_proba(test_x)
            if timed:
                print("Time taken to predict: ")
                print(time.process_time() - start)
                start = time.process_time()
            
            filter_test = np.nonzero(np.max(model_preds_probs_test, axis = 1) > threshold)
            filtered_probs_test = model_preds_probs_test[filter_test]
            if len(model_preds_probs_test) > len(filtered_probs_test):
                print(f"Removed {len(model_preds_probs_test)-len(filtered_probs_test)} estimates")
            
            model_preds_test = np.argmax(filtered_probs_test, axis = 1)
            mapped_preds_test = np.array([classes[x] for x in model_preds_test])
            filtered_y_test = test_y.take(filter_test[0])
            
            statistics[j][0] = statistics[j][0] + np.sum(mapped_preds_train == filtered_y_train) / num_splits
            statistics[j][1] = statistics[j][1] + np.sum(mapped_preds_test == filtered_y_test) / num_splits
            if timed:
                print("Time taken to score: ")
                print(time.process_time() - start)
            
            compute_tss(model, filtered_y_train, mapped_preds_train, Y.unique(), "Training", i, f"{species_name}_{animal_id}")
            compute_tss(model, filtered_y_test, mapped_preds_test, Y.unique(), "Testing", i, f"{species_name}_{animal_id}")
        i = i+1
    return statistics

Keras model specifications

In [10]:
def test_model(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(input_cols * time_step,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name = "BaseNN")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [11]:
def test_model_species(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=((input_cols-1) * time_step + 1,))
    species = keras.layers.Lambda(lambda x: x[:, (input_cols-1) * time_step])(inputs)
    data = keras.layers.Lambda(lambda x: x[:, :(input_cols-1) * time_step])(inputs)
    embed = keras.layers.Embedding(13, 32)(species)
    y = keras.layers.Concatenate()([data, embed])
    x = keras.layers.Dense(256, activation="relu")(y)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name = "BaseNNSpecies")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [12]:
def lstm_model(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(time_step, input_cols))
    x = keras.layers.LSTM(64, recurrent_dropout = .1, dropout = .1)(inputs)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name = "LSTM")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [13]:
def lstm_model_species(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(time_step, input_cols))
    species = keras.layers.Lambda(lambda x: x[:, 0, input_cols-1])(inputs)
    data = keras.layers.Lambda(lambda x: x[:, :, :input_cols-1])(inputs)
    embed = keras.layers.Embedding(13, 32)(species)
    x = keras.layers.LSTM(64, recurrent_dropout = .1, dropout = .1)(data) #Only use the recurrent connection on the actual time data
    y = keras.layers.Concatenate()([x, embed])
    x = keras.layers.Dense(64, activation="relu")(y)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name = "LSTMSpecies")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [14]:
def gru_model(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(time_step, input_cols))
    x = keras.layers.GRU(64, recurrent_dropout = .1, dropout = .1)(inputs)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name = "GRU")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [15]:
def gru_model_species(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(time_step, input_cols))
    species = keras.layers.Lambda(lambda x: x[:, 0, input_cols-1])(inputs)
    data = keras.layers.Lambda(lambda x: x[:, :, :input_cols-1])(inputs)
    embed = keras.layers.Embedding(13, 32)(species)
    x = keras.layers.GRU(64, recurrent_dropout = .1, dropout = .1)(data)
    y = keras.layers.Concatenate()([x, embed])
    x = keras.layers.Dense(64, activation="relu")(y)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name = "GRUSpecies")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [16]:
def cnn_model(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(time_step, input_cols))
    x = keras.layers.Conv1D(filters=32, kernel_size=10, padding="same")(inputs)

    x = keras.layers.Conv1D(filters=32, kernel_size=5, padding="same")(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name = "CNN")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [17]:
def cnn_model_species(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(time_step, input_cols))
    species = keras.layers.Lambda(lambda x: x[:, 0, input_cols-1])(inputs)
    data = keras.layers.Lambda(lambda x: x[:, :, :input_cols-1])(inputs)
    embed = keras.layers.Embedding(13, 32)(species)
    
    x = keras.layers.Conv1D(filters=32, kernel_size=3, padding="same")(data)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    
    x = keras.layers.Conv1D(filters=32, kernel_size=3, padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    
    #May also want to try max pooling and pooling along each step
    x = keras.layers.GlobalAveragePooling1D()(x)
    y = keras.layers.Concatenate()([x, embed])
    
    x = keras.layers.Dense(64, activation="relu")(y)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name = "CNNSpecies")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [18]:
def transformer_model(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(time_step, input_cols))
    
    pos_embed = keras_hub.layers.PositionEmbedding(time_step)(inputs)
    inputs_embed = inputs + pos_embed
    
    # Attention and Normalization
    x = keras.layers.MultiHeadAttention(
      key_dim=64, num_heads=8, dropout=.2
    )(inputs_embed, inputs_embed)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs_embed
    
    # Feed Forward Part
    x = keras.layers.Conv1D(filters=32, kernel_size=1, activation="relu")(res)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Conv1D(filters=inputs_embed.shape[-1], kernel_size=1)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x_res = x + res

    #x = keras.layers.MultiHeadAttention(
    #  key_dim=64, num_heads=8, dropout=.2)(x_res, x_res)
    #x = keras.layers.Dropout(.2)(x)
    #x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    #res = x + x_res
    
    # Feed Forward Part
    #x = keras.layers.Conv1D(filters=32, kernel_size=1, activation="relu")(res)
    #x = keras.layers.Dropout(.2)(x)
    #x = keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    #x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    #x = x + res
    
    x = keras.layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name = "Transformer")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

In [19]:
def transformer_model_species(input_cols = 69, time_step = 4, num_behaviors = 14):
    inputs = keras.Input(shape=(time_step, input_cols))
    
    species = keras.layers.Lambda(lambda x: x[:, :, input_cols-1])(inputs)
    data = keras.layers.Lambda(lambda x: x[:, :, :input_cols-1])(inputs)
    species_embed = keras.layers.Embedding(13, 32)(species)
    
    
    pos_embed = keras_hub.layers.PositionEmbedding(time_step)(data)
    data_embed = data + pos_embed
    y = keras.layers.Concatenate()([data_embed, species_embed])
    
    # Attention and Normalization
    x = keras.layers.MultiHeadAttention(
      key_dim=64, num_heads=8, dropout=.2
    )(y, y)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + y
    
    # Feed Forward Part
    x = keras.layers.Conv1D(filters=64, kernel_size=1, activation="relu")(res)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Conv1D(filters=y.shape[-1], kernel_size=1)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x = x + res
    
    x = keras.layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(.2)(x)
    outputs = keras.layers.Dense(num_behaviors, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name = "TransformerSpecies")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
    return model

Dataset creation and splitting

In [20]:
def make_target_group_splits(ids, target_ids = None): 
    #Ids are the groups to split on
    #Target id is the group that will remain in the testing set
    train_index = []
    test_index = []
    if target_ids == None:
        target_ids = ids.unique()
    for animal in target_ids:
        train_index.append(ids[ids != animal].index.tolist())
        test_index.append(ids[ids == animal].index.tolist())
      
    return list(zip(target_ids, train_index, test_index))

In [21]:
def make_splits(ids, num_splits = 5, frac_train = None):
    #Default behavior is StratifiedKFold 
    #Only usable with frac_train <= .5
    splits = []
    if frac_train == None:
        skf = model_selection.StratifiedKFold(n_splits = num_splits)
        for train, test in skf.split(np.zeros(len(ids.index)), ids):
            splits.append((-1, train, test))
        return splits
    if(frac_train > .5):
        print("make_splits only works with frac_train < .5")
        return
    skf = model_selection.StratifiedKFold(n_splits=int(1/frac_train))
    num_splits = min(int(1/frac_train), num_splits)
    indices = skf.split(np.zeros(len(ids.index)), ids)
    for i in range(num_splits):
        train, test = indices.__next__()
        splits.append((-1, test, train))
    return splits

In [22]:
def make_dataset(X, Y, time_step, ids):
    dummified = pd.get_dummies(Y)
    dummified = dummified.reindex(sorted(dummified.columns), axis=1)
    tempX = X[ids == ids.unique()[0]]
    tempDummy = dummified[ids == ids.unique()[0]]
    data_set = tf.keras.preprocessing.timeseries_dataset_from_array(np.array(tempX), np.array(tempDummy.reset_index(), dtype = int), sequence_length= time_step, sequence_stride=1, batch_size = None)
    for id in segment_ids.unique()[1:]:
        tempX = X[ids == id]
        tempDummy = dummified[ids == id]
        data_set = data_set.concatenate(tf.keras.preprocessing.timeseries_dataset_from_array(np.array(tempX), np.array(tempDummy.reset_index(), dtype = int), sequence_length= time_step, sequence_stride=1, batch_size = None))
    del tempX
    del tempDummy
    del dummified
    gc.collect()
    return data_set

In [23]:
data_set = make_dataset(X, Y, sample_rate * time_window, segment_ids)

2026-02-06 15:49:16.335430: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [24]:
num_splits = 4 #Overriden if splits are provided directly. Using this will stratify based on Y passed to training code
frac_test = .1
splits = [] #Zipped list of split id, train, test indicies. Split id is only used for reference purposes and can be set to any value

#splits = make_splits(animal_ids, 4)

#for animal in animal_ids.unique(): #Used to produce a 1/10 training amount
#    animal_index = animal_ids[animal_ids == animal].index.tolist()
#    temp = Y.iloc[animal_index]
#    splits_temp = make_splits(temp, num_splits, frac_test)
#    splits.append((animal, temp.iloc[splits_temp[0][1]].index.to_numpy(), temp.iloc[splits_temp[0][2]].index.to_numpy()))

splits = make_splits(Y + animal_ids.apply(str), 2, .1)

#splits = make_target_group_splits(animal_ids)
#splits = make_target_group_splits(animal_ids, [4])

In [25]:
svm_model = svm.LinearSVC()
clf_model = calibration.CalibratedClassifierCV(svm_model)
knn_model = neighbors.KNeighborsClassifier(n_neighbors = 100, n_jobs = -1)
rf_model = ensemble.RandomForestClassifier(1000, n_jobs = -1, min_samples_leaf = 100)
log_model = linear_model.LogisticRegression(solver = "saga")
#stats_animal_based_splits = train_models([log_model], data_set, Y, len(X.columns), splits = splits, num_splits = num_splits, time_step = sample_rate * time_window, timed = False, species_name = sample_name, species = use_species, species_categories = animal_ids.unique())

In [26]:
#Note that models that have different input size (2-D vs 3-D) cannot be trained together
train_models_keras([test_model_species], data_set, Y, len(X.columns), splits = splits, num_splits = num_splits, epochs = 30, time_step = sample_rate * time_window, timed = False, batch_size = 256, species = use_species, species_name = sample_name, cache = True, val_freq = 10000, transfer = transfer)
#train_models_keras([gru_model, transformer_model], data_set, Y, len(X.columns), splits = splits, num_splits = num_splits, epochs = 100, time_step = sample_rate * time_window, timed = False, batch_size = 256, species = use_species, species_name = sample_name, cache = True, val_freq = 10000000, transfer = transfer)

Epoch 1/30
163/163 - 72s - 439ms/step - categorical_accuracy: 0.6451 - loss: 0.8924
Epoch 2/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


163/163 - 2s - 14ms/step - categorical_accuracy: 0.7624 - loss: 0.6276
Epoch 3/30
163/163 - 2s - 12ms/step - categorical_accuracy: 0.7883 - loss: 0.5501
Epoch 4/30
163/163 - 2s - 12ms/step - categorical_accuracy: 0.8009 - loss: 0.5057
Epoch 5/30
163/163 - 2s - 11ms/step - categorical_accuracy: 0.8145 - loss: 0.4675
Epoch 6/30
163/163 - 2s - 11ms/step - categorical_accuracy: 0.8275 - loss: 0.4293
Epoch 7/30
163/163 - 2s - 11ms/step - categorical_accuracy: 0.8372 - loss: 0.4055
Epoch 8/30
163/163 - 2s - 11ms/step - categorical_accuracy: 0.8471 - loss: 0.3755
Epoch 9/30
163/163 - 2s - 10ms/step - categorical_accuracy: 0.8589 - loss: 0.3522
Epoch 10/30
163/163 - 2s - 10ms/step - categorical_accuracy: 0.8688 - loss: 0.3307
Epoch 11/30
163/163 - 2s - 10ms/step - categorical_accuracy: 0.8750 - loss: 0.3148
Epoch 12/30
163/163 - 2s - 10ms/step - categorical_accuracy: 0.8840 - loss: 0.2905
Epoch 13/30
163/163 - 2s - 10ms/step - categorical_accuracy: 0.8897 - loss: 0.2801
Epoch 14/30
163/163 - 2

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


146/146 - 2s - 11ms/step - categorical_accuracy: 0.6097 - loss: 1.1435
Epoch 3/15
146/146 - 2s - 11ms/step - categorical_accuracy: 0.6144 - loss: 1.0980
Epoch 4/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6237 - loss: 1.0610
Epoch 5/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6238 - loss: 1.0399
Epoch 6/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6284 - loss: 1.0323
Epoch 7/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6318 - loss: 1.0191
Epoch 8/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6329 - loss: 1.0089
Epoch 9/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6363 - loss: 1.0019
Epoch 10/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6347 - loss: 0.9979
Epoch 11/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6392 - loss: 0.9944
Epoch 12/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6417 - loss: 0.9866
Epoch 13/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6419 - loss: 0.9840
Epoch 14/15
146/146 - 1s - 9ms/st

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


162/162 - 2s - 10ms/step - categorical_accuracy: 0.7356 - loss: 0.6872
Epoch 3/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.7658 - loss: 0.5957
Epoch 4/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.7851 - loss: 0.5453
Epoch 5/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.8025 - loss: 0.5020
Epoch 6/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.8133 - loss: 0.4684
Epoch 7/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.8249 - loss: 0.4354
Epoch 8/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.8394 - loss: 0.4036
Epoch 9/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.8508 - loss: 0.3757
Epoch 10/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.8602 - loss: 0.3558
Epoch 11/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.8687 - loss: 0.3314
Epoch 12/30
162/162 - 2s - 10ms/step - categorical_accuracy: 0.8743 - loss: 0.3160
Epoch 13/30
162/162 - 2s - 11ms/step - categorical_accuracy: 0.8852 - loss: 0.2947
Epoch 14/30
162/162 - 2

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


146/146 - 1s - 9ms/step - categorical_accuracy: 0.6234 - loss: 1.1382
Epoch 3/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6272 - loss: 1.0880
Epoch 4/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6266 - loss: 1.0612
Epoch 5/15
146/146 - 1s - 10ms/step - categorical_accuracy: 0.6281 - loss: 1.0441
Epoch 6/15
146/146 - 1s - 10ms/step - categorical_accuracy: 0.6322 - loss: 1.0197
Epoch 7/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6309 - loss: 1.0181
Epoch 8/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6330 - loss: 1.0034
Epoch 9/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6381 - loss: 0.9924
Epoch 10/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6420 - loss: 0.9842
Epoch 11/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6437 - loss: 0.9783
Epoch 12/15
146/146 - 1s - 9ms/step - categorical_accuracy: 0.6426 - loss: 0.9765
Epoch 13/15
146/146 - 1s - 10ms/step - categorical_accuracy: 0.6451 - loss: 0.9677
Epoch 14/15
146/146 - 1s - 9ms/s

[[40886.0,
  227455.0,
  Empty DataFrame
  Columns: [Stationary, Walking, Swimming, Grooming, Eating, Shaking, Social]
  Index: [],
  Empty DataFrame
  Columns: [Stationary, Walking, Swimming, Grooming, Eating, Shaking, Social]
  Index: []]]